<a href="https://colab.research.google.com/github/Manggy94/PokerBrain/blob/master/Hands_Recorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This notebook is intended to record Hands from .txt files of different poker rooms, especially WINAMAX. To do so we'll use special objects from custom python modules: FileParser and HandConverter. Doc can be found on GitHub: https://github.com/Manggy94/PokerBrain.

#Preparation steps

First, We have to mount drive and verify its content, to add it to our path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then we can import necessary libraries and modules



In [ ]:
!pip install treys
import sys
import os
sys.path.append('/content/drive/MyDrive/PokerBrain/PokerBrain')
from preprocessor import Preprocessor
import numpy as np
import pandas as pd
import datetime
import multiprocessing

In [ ]:
os.getcwd()

'/content'

In [ ]:
if os.getcwd() != "/content/drive/MyDrive/PokerBrain/PokerBrain":
  os.chdir(f"{os.getcwd()}/drive/MyDrive/PokerBrain/PokerBrain")

In [ ]:
pp = Preprocessor()

In [ ]:
def register_hands(parse_all: bool=False):
  files = pp.conv.parser.get_holdem_game_files("history")
  if parse_all:
    stored_files = np.array([])
  else:
    stored_files = np.array([filename.replace(".csv","") for filename in os.listdir(f"Histories/Hands")])
  pending = np.array([x for x in files if x.replace(".txt", "") not in stored_files])
  print(pending.size)
  count_files, count_stored = files.size, stored_files.size
  total = pending.size
  print(f"There are {total} Files to register")
  count=0
  actual_count = 0
  for file_ in pending:
    print(file_)
    count+=1
    np_hands = pp.conv.parser.parse_file(file_)
    print(np_hands)
    np_hands = pp.conv.filter(np_hands)
    if np_hands.size > 0:
      print(f"File n°{count}: {np_hands.size} hands")
      actual_count+=1
      hands = pp.conv.build_hands(np_hands)
      hands.to_csv(f"{os.getcwd()}/Histories/Hands/{file_.replace('.txt', '')}.csv")
    else:
      print(f"File n°{count}: Empty")
    print(f"({count}/{total}): {np.round(count/total*100, 2)}%")
  print(f"{actual_count}/{total} files have been registered with at least one hand")


In [ ]:
register_hands()

26
There are 26 Files to register
20220308_Aalen 08_real_holdem_no-limit.txt
[None None None None None None None None None None None]
File n°1: Empty
(1/26): 3.85%
20150904_Aberdeen 12_real_holdem_no-limit.txt
[None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None]
File n°2: Empty
(2/26): 7.69%
20150910_Melbourne 03_real_holdem_no-limit.txt
[None None None None None None None None None None None None None None
 None]
File n°3: Empty
(3/26): 11.54%
20150904_Nice 06_real_holdem_no-limit.txt
[None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None]
File n°4: Empty
(4/26): 15.38%
20150919_Nice 23_real_holdem_no-limit.txt
[None None None None None None None None None None None None None None
 Non

In [ ]:
stored_files = np.array([filename.replace(".csv","") for filename in os.listdir(f"Histories/Hands")])
data = pd.concat([pd.read_csv(f"{os.getcwd()}/Histories/Hands/{file}.csv", index_col=0) for file in stored_files], ignore_index=True)

In [ ]:
data.head()

,hand,hand_id,tour_id,table_id,level,current_street,bb,ante,max_pl,buyin,...,Card_4,Card_4_rank,Card_4_suit,is_rainbow,is_triplet,is_monotone,has_pair,has_straightdraw,has_gutshot,has_flushdraw
0,Hand_id: 552788065093943321-1-1441879205\n\nTo...,552788065093943321-1-1441879205,128706001,24,0,SD,100.0,0.0,9,5.0,...,8d,8,d,False,False,False,False,True,True,True
1,Hand_id: 552788065093943321-2-1441879289\n\nTo...,552788065093943321-2-1441879289,128706001,24,0,T,100.0,0.0,9,5.0,...,NaN,None,None,True,False,False,False,True,True,False
2,Hand_id: 552788065093943321-3-1441879348\n\nTo...,552788065093943321-3-1441879348,128706001,24,0,SD,150.0,0.0,9,5.0,...,5h,5,h,False,False,False,False,True,True,True
3,Hand_id: 552788065093943321-4-1441879424\n\nTo...,552788065093943321-4-1441879424,128706001,24,0,T,150.0,0.0,9,5.0,...,NaN,None,None,True,False,False,True,False,False,False
4,Hand_id: 552788065093943321-5-1441879511\n\nTo...,552788065093943321-5-1441879511,128706001,24,0,SD,150.0,0.0,9,5.0,...,5d,5,d,True,False,False,False,True,True,False


In [ ]:
data.shape

(88185, 591)

In [ ]:
data.to_csv(f"{os.getcwd()}/Data/combo_data_tab.csv")

In [ ]:
interest_pos = ["UTG_", "HJ_", "CO_", "BTN_", "SB_", "BB_"]

In [ ]:
from API.listings import str_hands

In [ ]:
dico = {}
for x in str_hands:
  dico[x] = f"{x}"
dico["22.0"] = "22"
dico["33.0"] = "33"
dico["44.0"] = "44"
dico["55.0"] = "55"
dico["66.0"] = "66"
dico["77.0"] = "77"
dico["88.0"] = "88"
dico["99.0"] = "99"
dico["None"] = "None"
card_cols = [x for x in data.columns if "Card" in x]
hero_cols = [x for x in data.columns if "hero" in x]
flop_cols = [x for x in data.columns if "is" in x or "has" in x]

In [ ]:
to_concat =[]
for pos in interest_pos:
  n = len(pos)
  cols = [x for x in data.columns if pos in x ]
  new_cols = [x[n:] for x in cols]
  cols = ["level", "bb", "max_pl", "current_street"]+ hero_cols + cols + card_cols + flop_cols 
  new_cols = ["level", "bb", "max_pl", "current_street"] + hero_cols + new_cols + card_cols + flop_cols + ["position"]
  data_tab = data[cols].to_numpy()
  position = np.ones((data.shape[0],1)).astype(str)
  position[:] = pos[:-1]
  data_tab = np.concatenate((data_tab, position), axis=1)
  concat_df = pd.DataFrame(columns=new_cols, data=data_tab)
  to_concat.append(concat_df)
obs_data= pd.concat(to_concat).reset_index(drop=True)
obs_data["hand"] = obs_data["hand"].map(dico)
obs_data["hero_hand"] = obs_data["hero_hand"].map(dico)
#del(to_concat, n, new_cols, cols, position, concat_df, data_tab, dico)

In [ ]:
obs_data

,level,bb,max_pl,current_street,hero_combo,hero_hand,hero_first_suit,hero_second_suit,hero_first_rank,hero_second_rank,...,Card_4_rank,Card_4_suit,is_rainbow,is_triplet,is_monotone,has_pair,has_straightdraw,has_gutshot,has_flushdraw,position
0,0,100.0,9,SD,As5s,A5s,s,s,A,5,...,8,d,False,False,False,False,True,True,True,UTG
1,0,100.0,9,T,8s7s,87s,s,s,8,7,...,None,None,True,False,False,False,True,True,False,UTG
2,0,150.0,9,SD,6d3c,63o,d,c,6,3,...,5,h,False,False,False,False,True,True,True,UTG
3,0,150.0,9,T,Qc5d,Q5o,c,d,Q,5,...,None,None,True,False,False,True,False,False,False,UTG
4,0,150.0,9,SD,KcQh,KQo,c,h,K,Q,...,5,d,True,False,False,False,True,True,False,UTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529105,3,300.0,6,R,Kc8d,K8o,c,d,K,8,...,Q,c,True,False,False,False,True,True,False,BB
529106,3,300.0,6,PF,Tc8h,T8o,c,h,T,8,...,None,None,None,None,None,None,None,None,None,BB
529107,3,300.0,6,F,QdJc,QJo,d,c,Q,J,...,None,None,True,False,False,False,True,True,False,BB
529108,4,350.0,6,R,Ad5s,A5o,d,s,A,5,...,4,c,False,False,False,False,True,True,True,BB


In [ ]:
filter1 =~ obs_data["hand"].isin(["None", np.NaN])
shown_hands = obs_data[filter1].reset_index(drop=True)
filter2 = shown_hands["max_pl"] <7
short_handed = shown_hands[filter2].reset_index(drop=True)
filter3 = short_handed["current_street"] == "SD"
WTSD = short_handed[filter3].reset_index(drop=True)
filter4 = short_handed["hero_position"] != short_handed["position"]
training_hands = short_handed[filter4].reset_index(drop=True)

In [ ]:
features, hand_labels, combo_labels = shown_hands.drop(["hand", "combo"], axis=1), shown_hands["hand"], shown_hands["combo"]

In [ ]:
features.shape, hand_labels.shape, combo_labels.shape

((135710, 101), (135710,), (135710,))

In [ ]:
filter = features["hero_position"] != features["position"]
training_features = features[filter].reset_index(drop=True)
training_combos=combo_labels[filter].reset_index(drop=True)
training_hands=hand_labels[filter].reset_index(drop=True)

In [ ]:
features.to_csv(f"{os.getcwd()}/Data/features_tab.csv")
features

,level,bb,max_pl,current_street,hero_combo,hero_hand,hero_first_suit,hero_second_suit,hero_first_rank,hero_second_rank,...,Card_4_rank,Card_4_suit,is_rainbow,is_triplet,is_monotone,has_pair,has_straightdraw,has_gutshot,has_flushdraw,position
0,0,150.0,9,SD,6d3c,63o,d,c,6,3,...,5,h,False,False,False,False,True,True,True,UTG
1,0,150.0,9,SD,KcQh,KQo,c,h,K,Q,...,5,d,True,False,False,False,True,True,False,UTG
2,0,300.0,9,SD,As6h,A6o,s,h,A,6,...,4,c,False,False,False,False,True,True,True,UTG
3,0,300.0,9,SD,9s8c,98o,s,c,9,8,...,2,h,False,False,False,False,True,True,True,UTG
4,0,100.0,9,PF,Ts3c,T3o,s,c,T,3,...,None,None,None,None,None,None,None,None,None,UTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135705,13,1600.0,6,SD,Tc9s,T9o,c,s,T,9,...,K,s,True,False,False,False,True,True,False,BB
135706,1,200.0,6,PF,6h4c,64o,h,c,6,4,...,None,None,None,None,None,None,None,None,None,BB
135707,1,200.0,6,SD,Ks3s,K3s,s,s,K,3,...,6,s,True,False,False,False,True,True,False,BB
135708,2,250.0,6,PF,AsJs,AJs,s,s,A,J,...,None,None,None,None,None,None,None,None,None,BB


In [ ]:
training_features.to_csv(f"{os.getcwd()}/Data/training_features.csv")
training_features

,level,bb,max_pl,current_street,hero_combo,hero_hand,hero_first_suit,hero_second_suit,hero_first_rank,hero_second_rank,...,Card_4_rank,Card_4_suit,is_rainbow,is_triplet,is_monotone,has_pair,has_straightdraw,has_gutshot,has_flushdraw,position
0,0,150.0,9,SD,6d3c,63o,d,c,6,3,...,5,h,False,False,False,False,True,True,True,UTG
1,0,300.0,9,SD,9s8c,98o,s,c,9,8,...,2,h,False,False,False,False,True,True,True,UTG
2,0,150.0,9,SD,6c3s,63o,c,s,6,3,...,4,c,False,False,False,False,True,True,True,UTG
3,0,400.0,9,SD,Td4s,T4o,d,s,T,4,...,7,c,True,False,False,False,True,True,False,UTG
4,0,50.0,6,SD,3s2d,32o,s,d,3,2,...,Q,h,True,False,False,False,True,True,False,UTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53478,7,600.0,6,SD,Js7c,J7o,s,c,J,7,...,9,d,True,False,False,False,True,True,False,BB
53479,8,700.0,6,SD,AdAc,AA,d,c,A,A,...,J,d,False,False,False,False,True,True,True,BB
53480,9,800.0,6,SD,Kd6c,K6o,d,c,K,6,...,T,c,True,False,False,False,False,True,False,BB
53481,12,1400.0,6,SD,9h3c,93o,h,c,9,3,...,A,d,True,False,False,False,True,True,False,BB


In [ ]:
hand_labels.to_csv(f"{os.getcwd()}/Data/hand_labels_tab.csv")
hand_labels

0         K9o
1         KQo
2         A6o
3         KJo
4         T3o
         ... 
135705    72o
135706    64o
135707    K3s
135708    AJs
135709    T8o
Name: hand, Length: 135710, dtype: object

In [ ]:
training_hands.to_csv(f"{os.getcwd()}/Data/training_hands.csv")
training_hands

0        K9o
1        KJo
2        KJs
3         33
4        A6o
        ... 
53478    KJs
53479     77
53480    K6o
53481    Q7o
53482    72o
Name: hand, Length: 53483, dtype: object

In [ ]:
combo_labels.to_csv(f"{os.getcwd()}/Data/combo_labels_tab.csv")
combo_labels

0         Kc9h
1         KcQh
2         As6h
3         KsJh
4         Ts3c
          ... 
135705    7c2s
135706    6h4c
135707    Ks3s
135708    AsJs
135709    Tc8h
Name: combo, Length: 135710, dtype: object

In [ ]:
training_combos.to_csv(f"{os.getcwd()}/Data/training_combos.csv")
training_combos

0        Kc9h
1        KsJh
2        KdJd
3        3h3d
4        Ah6d
         ... 
53478    KdJd
53479    7s7d
53480    Kc6h
53481    Qh7c
53482    7c2s
Name: combo, Length: 53483, dtype: object